Steps for processing input data (assume this is a csv file, or a path to a csv file).

In [ ]:
# Drop records with NaN for totalRent
df_val = df_val.dropna(subset=['totalRent'])

In [ ]:
# Drop text features and target variable
df_val_no_text = df_val.drop(columns=['description', 'facilities', 'totalRent'],axis=1)
df_val_no_text= label_encoding(df_val_no_text, cat_features)

In [1]:
# Drop columns

columns_1_drop = ['telekomHybridUploadSpeed', 'noParkSpaces', 'petsAllowed', 
           'thermalChar', 'numberOfFloors', 'heatingCosts', 'energyEfficiencyClass',
          'lastRefurbish', 'electricityBasePrice', 'electricityKwhPrice']

columns_2_drop = ['street','streetPlain', 'geo_bln', 'scoutId', 'date', 'geo_krs', 'houseNumber',
          'yearConstructedRange', 'baseRentRange', 'noRoomsRange', 'livingSpaceRange',
          'telekomTvOffer', 'firingTypes']

num_features = ['serviceCharge', 'picturecount', 'pricetrend', 'telekomUploadSpeed', 
                'totalRent', 'yearConstructed', 'baseRent', 'livingSpace', 'noRooms', 
                'floor']

In [ ]:
# Drop baseRent

In [2]:
len(num_features)

10

In [ ]:
num_missing_cols = ['serviceCharge', 'yearConstructed', 'pricetrend', 'telekomUploadSpeed', 'floor']

# Fill by medians
# Apply this to df_val
for col in num_missing_cols:
    # For each row in df_val, if the value in the target column is missing,
    # fill it with the median from medians_by_regio2 where 'regio2' matches.
    df_val[col] = df_val.apply(
        lambda row: medians_by_regio2.loc[medians_by_regio2['regio2'] == row['regio2'], col].values[0]
        if pd.isnull(row[col]) else row[col], axis=1)
    
# Apply this to df_test
for col in num_missing_cols:
    # For each row in df_test, if the value in the target column is missing,
    # fill it with the median from medians_by_regio2 where 'regio2' matches.
    df_test[col] = df_test.apply(
        lambda row: medians_by_regio2.loc[medians_by_regio2['regio2'] == row['regio2'], col].values[0]
        if pd.isnull(row[col]) else row[col], axis=1)

# Filling missing text values with 'keine Beschreibung' for train set
df_val['description'] = df_val['description'].fillna('keine Beschreibung')
df_val['facilities'] = df_val['facilities'].fillna('keine Beschreibung')

In [ ]:
# For no_text set
df_train_no_text = df_train.drop(columns=['description', 'facilities', 'totalRent'],axis=1)
df_train_no_text

In [ ]:
# Fill with trained imputers
def fill_with_trained_imputer(test_df, clf, column, missing_label):
    # Prepare test data with missing values
    X_missing_test = test_df[test_df[column] == missing_label].drop(column, axis=1)

    # Predict and fill missing values in the test set using the trained classifier
    if not X_missing_test.empty:
        predicted_values_test = clf.predict(X_missing_test)
        test_df.loc[test_df[column] == missing_label, column] = predicted_values_test

    print(f"Imputation completed for {column} in test set.")

In [ ]:
fill_with_trained_imputer(df_val_no_text, clf_heatingType, 'heatingType', -1)

fill_with_trained_imputer(df_val_no_text, clf_condition, 'condition', -1)

fill_with_trained_imputer(df_val_no_text, clf_typeOfFlat, 'typeOfFlat', -1)

fill_with_trained_imputer(df_val_no_text, clf_interiorQual, 'interiorQual', -1)

# Add the target variable to df_val_no_text
df_val_no_text['totalRent'] = df_val['totalRent']
# Modify num_features in df_val with standard scaler
df_val[num_features] = df_val_no_text[num_features]
# Fill missing values and label encoder in df_val
df_val[cat_features] = df_val_no_text[cat_features]

1%
Top 10 important features:  ['livingSpace', 'pricetrend', 'serviceCharge', 'geo_plz', 'facilities_vec_1', 'description_vec_4', 'yearConstructed', 'regio3', 'picturecount', 'description_vec_6']

2%
Top 10 important features:  ['geo_plz', 'livingSpace', 'serviceCharge', 'regio2', 'pricetrend', 'yearConstructed', 'regio3', 'regio1', 'picturecount', 'condition']

Top 10 important features:  ['livingSpace' (high), 'serviceCharge' (high), 'pricetrend' (high), 'geo_plz' (high), 'yearConstructed', 'regio2', 'regio3', 'description_vec_3', 'description_vec_6', 'description_vec_8']
Mean valiation R2 score:  0.8244478048145927
validation: RMSE loss:  197.72233966186332
R2 score:  0.797261771694843

test: RMSE loss:  224.6605712104951
R2 score:  0.7935056412964735

 79% (new features)

 10%
